In [ ]:
# рис. 9
import numpy as np
import math #для ln()
import matplotlib.pyplot as plt
from numpy import  exp, log, pi, sqrt
import random
from numpy import linalg # для собственные значения матрицы
import scipy.special as sc # для гипергеометрической функции Mhyper

M=100 #кол-во усреднений для оценки КМ-цы
N=10 #длина выборки (число компонент сигнала)
f=0 #частота сигнала

#возвращает Массив случайных комплексных чисел <class 'numpy.ndarray'>
def rnorm(N,mu,sigma):
    arr = np.array([])
    for i in range(N):
        a=random.gauss(mu, sigma)
        b=random.gauss(mu, sigma)
        arr = np.append(arr , complex(a,b))
    return arr

def rnorm_1(N,mu,sigma):
    arr = np.array([])
    for i in range(N):
        a=random.gauss(mu, sigma)
        arr = np.append(arr , a)
    return arr
# Энтропия: детерминированный сигнал + собственный шум без флуктуаций 
def H0(A,f):
    R = np.zeros((M*N,N,N),dtype=np.complex128)#Корреляционная матрица пришедшего сигнала
    for i in range(N*M):
        S0 = np.array([])
        Noise =  rnorm(N,0,1)
        for k in range(N):
            S0 = np.append(S0 , Noise[k]+A*exp(2j*math.pi*f*k/N))
        #транспонированная матрица
        S=np.array([S0]).T
        #комплексно-сопряженная матрица
        S_h= np.conj(S0)
        R[i]=S*S_h 
    RRR  = np.zeros((N,N),dtype=np.complex128)
    for i in range(N*M):
        RRR=RRR+R[i]
    RRR  = RRR/(N*M) 
    #собственные значения матрицы RRR
    L=linalg.eig(RRR)[0] 
    EV=L
    sum_EV=0
    for i in range(N): sum_EV+=abs(EV[i])
    AN=EV/sum_EV
    E=0
    for i in range(N): E+= - abs(AN[i])*math.log(abs(AN[i]))
    return E

def H0_gr(A,f):
    gr=[]
    for i in range(len(A)): 
        gr.append(H0(A[i], f))
    return gr

# Энтропия: детерминированный сигнал со случайной частотой + собственный шум
# sigma_f - это СКО случайных частот, распределенных по нормальному закону
def H8(A, sigma_f):
    R = np.zeros((M*N,N,N),dtype=np.complex128) # Корреляционная матрица пришедшего сигнала
    for i in range(N*M):
        S0 = np.array([])
        Noise =  rnorm(N,0,1)
        w = rnorm_1(N,0, sigma_f)  #случайные частоты, распределенные по нормальному закону
        for k in range(N):
            S0 = np.append(S0 , Noise[k]+A*exp(1j*(2*math.pi*(f+ w[k])*k/N )))
        #транспонированная матрица
        S=np.array([S0]).T
        #комплексно-сопряженная матрица
        S_h= np.conj(S0)
        R[i]=S*S_h 
    RRR  = np.zeros((N,N),dtype=np.complex128)
    for i in range(N*M):
        RRR=RRR+R[i]
    RRR  = RRR/(N*M)
    #собственные значения матрицы RRR
    L=linalg.eig(RRR)[0] 
    EV=L
    sum_EV=0
    for i in range(N): sum_EV+=abs(EV[i])
    AN=EV/sum_EV
    E=0
    for i in range(N): E+= - abs(AN[i])*math.log(abs(AN[i]))
    return E


def H8_gr(A,sigma_f):
    gr=[]
    for i in range(len(A)): 
        gr.append(H8(A[i], sigma_f))
    return gr


del_f = 1
def c(n):
    return (n*del_f)/(2*sqrt(2*pi))
def F1(q_0n):
    Mhyper=sc.hyp1f1(1/2, 1, -q_0n**2/2) 
    return Mhyper
    
q01=4.15
q02=2.285
q03=1.55
print("1* def_f: \n" , c(1), F1(q01))
print("2* def_f: \n" , c(2), F1(q02))
print("3* def_f: \n" , c(3), F1(q03))

a1=H0(q01, f)
a2=H0(q02, f)
a3=H0(q03, f)

def E01(A,q01): 
    a11=[]
    for i in range(len(A)):
        a11.append(a1)
    return a11

def E02(A,q02): 
    a22=[]
    for i in range(len(A)):
        a22.append(a2)
    return a22

def E03(A,q03): 
    a33=[]
    for i in range(len(A)):
        a33.append(a3)
    return a33

A = np.arange(0,10,0.1)
delta_f=1/N 

sigmas = [ 1, 2, 3] # СКО случайных частот
labels=['3','2', '1', '7', '6', '5']
colors=['b', 'g', 'c', 'b', 'g', 'c']

fig, ax = plt.subplots()

gr=H0_gr(A,1)/log(N)
ax.plot(A, gr,'red' , linestyle = '-.', label='4', linewidth=4)
plt.axis([0, 10, 0, 1])
xmin, xmax, ymin, ymax = plt.axis()
plt.text(xmax+0.1, min(gr), '4', color ='red' , fontsize = 0, fontweight='bold')

for i in range(len(sigmas)):
    gr=H8_gr(A, sigmas[i]*delta_f)/log(N)
    ax.plot(A, gr,colors[i], label=labels[i], linewidth=4 )
    plt.axis([0,10,0,1])
    xmin, xmax, ymin, ymax = plt.axis()
    x_text = xmax+0.1
    plt.text(x_text, min(gr), labels[i], color =colors[i] , fontsize = 20, fontweight='bold')

gr=E01(A,q01)/log(N)
ax.plot(A, gr,colors[3],linestyle = 'dashed', label=labels[3], linewidth=4 )
xmin, xmax, ymin, ymax = plt.axis()
x_text = xmin+0.4
plt.text(x_text, min(gr)+0.03, labels[3], color =colors[3] , fontsize = 20, fontweight='bold')

gr=E02(A,q02)/log(N)
ax.plot(A, gr,colors[4],linestyle = 'dashed', label=labels[4], linewidth=4 )
xmin, xmax, ymin, ymax = plt.axis()
x_text = xmin+0.4
plt.text(x_text, min(gr)+0.03, labels[4], color =colors[4] , fontsize = 20, fontweight='bold')

gr=E03(A,q03)/(log(N))
ax.plot(A, gr,colors[5],linestyle = 'dashed', label=labels[5], linewidth=4 )
xmin, xmax, ymin, ymax = plt.axis()
x_text = xmin+0.4
plt.text(x_text, min(gr)+0.03, labels[5], color =colors[5] , fontsize = 20, fontweight='bold')

plt.xlabel('q', color='black',fontsize = 'xx-large', style = 'italic', weight=600, loc='center')
plt.ylabel('H', color='black',fontsize = 'xx-large',style = 'italic', weight=600 , rotation=0, y=1.03, labelpad=20)

plt.tick_params(labelsize = 15)
#plt.legend(loc = 'upper right', fontsize = 15)
plt.rcParams["figure.figsize"] = (16, 12) 
plt.grid(True)
plt.show()